<a href="https://colab.research.google.com/github/poziryna84/Neural-Network-Models/blob/master/Head_rotation__Mobile__net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In this tutorial we are going to:

- having as input photoes of different head positions (its coordinates and angles) create a data architecture appropriate for mobile net neural net model;
- fit the model.

In [0]:
import os
import pandas as pd
import numpy as np
import glob
import re
from PIL import Image, ImageOps

In [0]:
path = '/content/drive/My Drive/head_rotation'

In [0]:
def img_df(image_path, shape):
    image = Image.open(image_path)
    image_resized = image.resize(shape, Image.ANTIALIAS)
    img_array = np.asarray(image_resized)
    return img_array

In [0]:
# Formar X e Y recorriendo las carpetas y ficheros
df = pd.DataFrame()
regex = r'(\W\d+)(\W\d+)'
jpg_list = []

for txt_path in glob.glob('/content/drive/My Drive/head_rotation/Person*/*.txt'):
    jpg_path = txt_path[:-3] + "jpg"
    jpg_list.append(jpg_path)
    df = df.append(pd.read_csv(txt_path, header=None).T)
    


In [0]:
df = df.drop(1, axis=1)


In [0]:
df["T"] = df[0].apply(lambda cell: int(re.findall(regex, cell)[0][0]))
df["P"] = df[0].apply(lambda cell: int(re.findall(regex, cell)[0][1]))


In [0]:
df["Image_path"] = jpg_list

In [0]:
df = df[~df.Image_path.apply(lambda x: '(' in x)]

In [0]:
df = df[~df.Image_path.apply(lambda x: ')' in x)]

In [0]:
df["Image"] = df['Image_path'].apply(lambda img: img_df(img, (224, 224)))

In [25]:
df.Image.head()[:2]

0    [[[22, 22, 22], [14, 14, 14], [70, 70, 70], [2...
0    [[[22, 22, 22], [15, 15, 15], [66, 66, 66], [1...
Name: Image, dtype: object

In [0]:
df = df.drop([0, "Image_path"], axis=1)
df.columns = ["X", "Y", "H", "W", "T", "P", "Image"]
df.X = df.X.astype(int)
df.Y = df.Y.astype(int)
df.H = df.H.astype(int)
df.W = df.W.astype(int)
df = df.reset_index().drop("index", axis=1)

In [27]:
df.head()

,X,Y,H,W,T,P,Image
0,206,235,108,73,-90,0,"[[[22, 22, 22], [14, 14, 14], [70, 70, 70], [2..."
1,311,192,72,97,-60,-90,"[[[22, 22, 22], [15, 15, 15], [66, 66, 66], [1..."
2,294,195,93,101,-60,-60,"[[[23, 23, 23], [14, 14, 14], [63, 63, 63], [1..."
3,305,197,82,102,-60,-75,"[[[20, 20, 20], [17, 17, 17], [61, 61, 61], [1..."
4,275,196,100,101,-60,-45,"[[[21, 21, 21], [14, 14, 14], [64, 64, 64], [1..."


In [0]:
# Extraer X e Y del df y normalizar
X = np.asarray(list(df["Image"]/255.))
Y = np.array(df[["X", "Y", "H", "W", "T", "P"]])/100.

In [29]:
X.shape

(2790, 224, 224, 3)

In [30]:
# Crear red neuronal
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.models import Model

Using TensorFlow backend.


In [31]:
n_classes = 6
base_model = MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x = Dense(1024,activation='relu')(x) #dense layer 2
x = Dense(512,activation='relu')(x) #dense layer 3

preds = Dense(n_classes,activation='linear')(x)

model=Model(inputs=base_model.input,outputs=preds)


/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '



17227776/17225924 [==============================] - 3s 0us/step


In [0]:
# Entrenar red neuronal
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33)

In [33]:
model.compile(optimizer = 'Adam',
              loss ='mean_squared_error',
              metrics = ['cosine_proximity'])

In [34]:
model.fit(X_train, Y_train, validation_data=[X_test, Y_test], epochs=15, verbose=2)

Train on 1869 samples, validate on 921 samples
Epoch 1/15
 - 32s - loss: 0.3911 - cosine_proximity: -9.4518e-01 - val_loss: 1.2096 - val_cosine_proximity: -9.8498e-01
Epoch 2/15
 - 24s - loss: 0.0188 - cosine_proximity: -9.9477e-01 - val_loss: 0.1386 - val_cosine_proximity: -9.9502e-01
Epoch 3/15
 - 25s - loss: 0.0115 - cosine_proximity: -9.9645e-01 - val_loss: 0.0532 - val_cosine_proximity: -9.9678e-01
Epoch 4/15
 - 25s - loss: 0.0084 - cosine_proximity: -9.9738e-01 - val_loss: 0.0086 - val_cosine_proximity: -9.9761e-01
Epoch 5/15
 - 25s - loss: 0.0080 - cosine_proximity: -9.9736e-01 - val_loss: 0.0105 - val_cosine_proximity: -9.9820e-01
Epoch 6/15
 - 25s - loss: 0.0055 - cosine_proximity: -9.9833e-01 - val_loss: 0.0142 - val_cosine_proximity: -9.9606e-01
Epoch 7/15
 - 25s - loss: 0.0061 - cosine_proximity: -9.9817e-01 - val_loss: 0.0082 - val_cosine_proximity: -9.9816e-01
Epoch 8/15
 - 25s - loss: 0.0071 - cosine_proximity: -9.9770e-01 - val_loss: 0.0146 - val_cosine_proximity: -9.98